In [1]:
import pandas as pd
import pyodbc as pc
from sqlalchemy import create_engine
import datetime as datetime
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',5000)

In [2]:
databaseName = 'sreeman_live'
username = 'sa'
password = 'Hemanth@123'
server = 'DESKTOP-L57QDTP\SQLEXPRESS'
driver= '{SQL Server Native Client 10.0}'
CONNECTION_STRING = 'DRIVER='+driver+';SERVER='+server+';DATABASE='+databaseName+';UID='+username+';PWD='+ password
#Create/Open a Connection to Microsoft's SQL Server
conn = pc.connect(CONNECTION_STRING)
#Close the Connection
#conn.close()

databaseName1 = 'suriya_live'
username1 = 'sa'
password1 = 'Hemanth@123'
server1 = 'DESKTOP-L57QDTP\SQLEXPRESS'
driver1= '{SQL Server Native Client 10.0}'

CONNECTION_STRING2 = 'DRIVER='+driver1+';SERVER='+server1+';DATABASE='+databaseName1+';UID='+username1+';PWD='+ password1
#Create/Open a Connection to Microsoft's SQL Server
conn1 = pc.connect(CONNECTION_STRING2)
#Close the Connection
#conn.close()

sql = "SELECT * FROM DBO.Sreeman_full_data_view"
df_sreem = pd.read_sql(sql,conn)

sql1 = "SELECT * FROM DBO.Suriya_full_data_view"
df_suri = pd.read_sql(sql1,conn1)

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:15: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:15: SyntaxWarning: invalid escape sequence '\S'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:4: SyntaxWarning: invalid escape sequence '\S'
  server = 'DESKTOP-L57QDTP\SQLEXPRESS'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:15: SyntaxWarning: invalid escape sequence '\S'
  server1 = 'DESKTOP-L57QDTP\SQLEXPRESS'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sreem = pd.read_sql(sql,conn)
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Othe

In [ ]:
# Set up the connections
databaseName = 'sreeman_live'
username = 'sa'
password = 'sa'
server = 'SREEMAN-NEW'
driver = '{SQL Server Native Client 10.0}'
CONNECTION_STRING = 'DRIVER=' + driver + ';SERVER=' + server + ';DATABASE=' + databaseName + ';UID=' + username + ';PWD=' + password

databaseName1 = 'suriya_live'
username1 = 'sa'
password1 = 'sa'
server1 = 'SREEMAN-NEW'
driver1 = '{SQL Server Native Client 10.0}'
CONNECTION_STRING2 = 'DRIVER=' + driver1 + ';SERVER=' + server1 + ';DATABASE=' + databaseName1 + ';UID=' + username1 + ';PWD=' + password1

# Create/Open connections
conn = pc.connect(CONNECTION_STRING)
conn1 = pc.connect(CONNECTION_STRING2)

# Execute SQL queries and create DataFrames
sql = "SELECT * FROM DBO.Sreeman_full_data_view"
df_sreem = pd.read_sql(sql, conn)

sql1 = "SELECT * FROM DBO.Suriya_full_data_view"
df_suri = pd.read_sql(sql1, conn1)

In [ ]:
#df_sreeman=df_sreeman
df_suriya=df_suri
df_sreeman=df_sreem

df_sreeman['Company']='Sreeman'
df_suriya['Company']='Suriya'

df = pd.concat([df_sreeman, df_suriya], ignore_index=True)


def date_ext_mon1(mon):
    datee = datetime.datetime.strptime(mon, "%Y-%m-%d")
    month = datetime.date(datee.year, datee.month, datee.day).strftime('%B')
    return month

def date_ext_year1(year):
    #print(year)
    datee = datetime.datetime.strptime(year, "%Y-%m-%d")
    Year = datee.year
    return Year

def date_ext_day1(day):
    datee = datetime.datetime.strptime(day, "%Y-%m-%d")
    Day = datee.day
    return Day

#this is for sreeman.
##this is to filter out the unit wise payment.
df['lr_hire_date']= df['lr_hire_date'].astype(str)
df['lr_hire_date']= df['lr_hire_date'].astype(str)
df['YEAR']=df['lr_hire_date'].apply(lambda x: date_ext_year1(x))
df['MONTH']=df['lr_hire_date'].apply(lambda x: date_ext_mon1(x))
df['DAY']=df['lr_hire_date'].apply(lambda x: date_ext_day1(x))

In [ ]:
df1=df

In [ ]:
df1=df1[(df1['lr_hire_date']>='2023-12-01') & (df1['lr_hire_date']<='2023-12-09')]

In [ ]:
df1.columns

In [ ]:
df1['from_branch'].unique()

In [ ]:
shipped_from='M/s. AQUAPUMP INDUSTRIES'
freight_billed_to='M/s. AQUAPUMP INDUSTRIES (IND)'
# dont use this while generating report for invoice for now
df1=df1[(df1['from_branch']==shipped_from) & (df1['to_branch']==freight_billed_to)]

In [ ]:
df2=df1.groupby(['Company','gdm_no','lr_hire_date','lr_number','lorry_reg_number','LR_party_invoice','from_branch','to_branch','LR_no_of_packages','LR_weight','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin'])['LR_Rate'].unique().reset_index().explode('LR_Rate')
df3=df2.groupby(['Company','gdm_no','lr_hire_date','lr_number','lorry_reg_number','from_branch','to_branch','LR_weight','LR_Rate','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin']).agg({'LR_no_of_packages':'sum'}).reset_index()
df4=df3[['Company','gdm_no','from_branch','to_branch','lr_hire_date','lr_number','lorry_reg_number','LR_no_of_packages','LR_weight','LR_Rate','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin']]
df4['stat_chg']=15
df4['total_amount']=round((df4['LR_weight']*df4['LR_Rate'])+df4['stat_chg'])
#df4=df4[['Company','gdm_no','from_branch','to_branch','lr_hire_date', 'lr_number', 'lorry_reg_number', 'LR_no_of_packages', 'LR_weight', 'LR_Rate','stat_chg', 'total_amount']]
df4['lr_hire_date'] = pd.to_datetime(df4['lr_hire_date'])
df4['lr_hire_date'] = df4['lr_hire_date'].dt.strftime('%d-%m-%Y')

In [ ]:
df4

In [ ]:
if df4['Company'].unique()[0]=='Sreeman':
    Company_Name='SREEMAN TRANSPORTS'
elif df4['Company'].unique()[0]=='Suriya':
    Company_Name='SURIYA CARRIERS'
else:
    print('Company not listed')

In [ ]:
df4.to_excel('taxinvoicereport.xlsx')

In [ ]:
consignor_gstin=df4['consignor_gstin'].unique()[0]
consignor_addr=df4['consignor_addr'].unique()[0]
consignor_pincode=df4['consignor_pincode'].unique()[0]
consignor_state=df4['consignor_state'].unique()[0]
from_city=df4['from_city'].unique()[0]


In [ ]:
consignee_gstin=df4['consignee_gstin'].unique()[0]
consignee_addr=df4['consignee_addr'].unique()[0]
consignee_pincode=df4['consignee_pincode'].unique()[0]
consignee_state=df4['consignee_state'].unique()[0]
to_city=df4['to_city'].unique()[0]


In [ ]:
consignor_addr_combined=consignor_addr.lower()+" "+from_city.upper()+" "+consignor_pincode
consignee_addr_combined=consignee_addr.lower()+" "+to_city.upper()+" "+consignee_pincode

In [ ]:
LR_no_of_packages=df4['LR_no_of_packages'].sum()
LR_weight=df4['LR_weight'].sum()
total_amount=df4['total_amount'].sum()

In [ ]:
total_amount

In [113]:
import inflect

In [114]:
total_amount=33186
def number_to_words(number):
    p = inflect.engine()
    words = p.number_to_words(number,andword="and")
    words = words.replace(',', '')  # Remove the comma
    return words

words = number_to_words(total_amount)
word_expand=words.capitalize() + " only"


In [115]:
word_expand


'Thirty-three thousand one hundred and eighty-six only'

In [116]:
CGST_RATE='2.5%'
SGST_RATE='2.5%'
IGST_RATE='5%'

In [117]:
total_amount=2930
CGST_AMOUNT=(2.5/100)*total_amount
CGST_AMOUNT=round(CGST_AMOUNT,2)
SGST_AMOUNT=(2.5/100)*total_amount
SGST_AMOUNT=round(CGST_AMOUNT,2)
TOTAL_AMOUNT_GST=CGST_AMOUNT+SGST_AMOUNT
IGST_AMOUNT=(5/100)*total_amount
IGST_AMOUNT=IGST_AMOUNT

In [118]:
SGST_AMOUNT,SGST_AMOUNT,TOTAL_AMOUNT_GST,IGST_AMOUNT

(73.25, 73.25, 146.5, 146.5)

In [127]:
df4['LR_Rate'].astype(int)

0    1
1    1
Name: LR_Rate, dtype: int32

0    0.01
1    0.01
Name: LR_Rate, dtype: float64

In [131]:
df4['LR_Rate'] = df4['LR_Rate'].astype(float) / 100
df4['LR_Rate'] = df4['LR_Rate'].round(2)

C:\Users\Dell\AppData\Local\Temp/ipykernel_9496/3363364578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['LR_Rate'] = df4['LR_Rate'].astype(float) / 100
C:\Users\Dell\AppData\Local\Temp/ipykernel_9496/3363364578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['LR_Rate'] = df4['LR_Rate'].round(2)


In [147]:
df4['LR_Rate']=df4['LR_Rate'].map("{:.2f}".format)

0    1.00
1    1.00
Name: LR_Rate, dtype: object

In [148]:
total_amount=4234234
"{:.2f}".format(total_amount)

'4234234.00'

In [2]:
var=127555.76

In [5]:
rounder_var=round(var)

In [15]:
round(rounder_var-var,2)

0.24

In [9]:
var1=127555.34

In [10]:
rounder_var1=round(var1)

In [12]:
rounder_var1

127555

In [14]:
round(rounder_var1-var1,2)

-0.34

In [18]:
import inflect

def number_to_words_indian(number):
    p = inflect.engine()

    # Set the Indian numbering system (assuming correct method):
    p.set_determiner_suffix("en_IN")  # Replace with the actual method if different

    # Convert number to words
    words = p.number_to_words(number, andword="and")

    # Remove the comma
    words = words.replace(',', '')

    return words

total_amount = 33186
words_indian = number_to_words_indian(total_amount)
word_expand_indian = words_indian.capitalize() + " only"

print(word_expand_indian)  # Should output in Indian system if configuration is correct


AttributeError: 'engine' object has no attribute 'set_determiner_suffix'

In [44]:
def number_to_words_indian(number):
    units = ["", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
    teens = ["ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen"]
    tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

    def convert_integer_part(number):
        words = []
        if number >= 10000000:
            crores = number // 10000000
            words.extend(convert_integer_part(crores))
            words.append("crore")
            number %= 10000000
        if number >= 100000:
            lakhs = number // 100000
            words.extend(convert_integer_part(lakhs))
            words.append("lakh")
            number %= 100000
        if number >= 1000:
            thousands = number // 1000
            words.extend(convert_integer_part(thousands))
            words.append("thousand")
            number %= 1000
        if number >= 100:
            hundreds = number // 100
            words.append(units[hundreds])
            words.append("hundred")
            number %= 100
        if number > 0:
            if number < 20:
                words.append(units[number])
            else:
                words.append(tens[number // 10])
                if number % 10 > 0:
                    words.append(units[number % 10])
        return words

    def convert_decimal_part(number):
        words = []
        decimal_digits = str(number).split(".")[1] if "." in str(number) else "0"
        if decimal_digits != "00" and decimal_digits != "0":
            words.append("and")
            decimal_in_words = convert_integer_part(int(decimal_digits))
            words.extend(decimal_in_words)
            words.append("paise")
        return words

    words = []
    integer_part, decimal_part = str(number).split(".") if "." in str(number) else (str(number), "0")
    words.extend(convert_integer_part(int(integer_part)))
    if decimal_part:  # Check if a decimal part exists
        words.extend(convert_decimal_part(float("0." + decimal_part)))

    return " ".join(words).capitalize() + " only"

# Example usage:
number = 452454
result = number_to_words_indian(number)
print(result)


Four lakh fifty two thousand four hundred fifty four only


In [ ]:
print(result) 